## RAG with Multi Data Source


In [69]:
### Wikipedia Tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [70]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250) 
# top_k_results=1 this dsiplays the top resukts from wikiapi as it is given 1 then first top reult will be the response
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)


wikipedia


In [71]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader(web_path="https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model='llama3.2'))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000218FDB088E0>, search_kwargs={})

In [72]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool=create_retriever_tool(retriever,name="langsmith_search",description="Search for informatioon about langsmith")

In [73]:
retrieval_tool.name

'langsmith_search'

In [74]:
### Arxiv Tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [75]:
### COMBINING ALL THE TOOLS

tools=[wiki,arxiv,retrieval_tool]

In [76]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for informatioon about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000218E8334550>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000218FDB088E0>, search_kwargs={})

In [77]:
from dotenv import load_dotenv

load_dotenv()
import os

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
from langchain_ollama import ChatOllama

llm=ChatOllama(model='llama3.2',temperature=0.3)

In [78]:
## Prompt Template

from langchain import hub

## get the prompt from hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages
print(type(prompt))


<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


In [79]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [80]:
# from langchain_core.prompts import ChatPromptTemplate
print(type(agent))
agent


<class 'langchain_core.runnables.base.RunnableSequence'>


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [81]:
## To run agent we need agent executor

from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

type(agent_executor)

langchain.agents.agent.AgentExecutor

In [82]:
agent_executor.invoke({"input":"Tell me about  Langsmith"})



> Entering new AgentExecutor chain...



Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Log your first trace​Tracing to LangSmith for LangChain usersThere is no need to use the LangSmith SDK directly if your application is built entirely on LangChain (either Python and JS).We've outlined a tracing guide specifically for LangChain users here.We provide multiple ways to log traces to LangSmith. Below, we'll highlight

description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input['postfix'], # Your AI system goes here    data=da

{'input': 'Tell me about  Langsmith',
 'output': 'Langsmith is a tool for building and deploying AI models. It provides a suite of APIs and tools for developers to create, train, and deploy machine learning models.\n\nHere are some key features of Langsmith:\n\n1. **Model Building**: Langsmith allows users to build and train machine learning models using a variety of algorithms and techniques.\n2. **Model Deployment**: Once a model is trained, Langsmith provides tools for deploying it in production environments.\n3. **Model Evaluation**: Langsmith includes a suite of evaluation tools that allow developers to assess the performance of their models.\n4. **Data Management**: Langsmith provides tools for managing and preparing data for use with machine learning models.\n\nLangsmith is designed to be used by developers who want to build and deploy AI models, but it can also be used by researchers and data scientists who want to create and train machine learning models.\n\nSome of the benefi

In [83]:
agent_executor.invoke({"input":"Tell me about  page 160 "})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'page 160'}`


Page: Tupolev Tu-160
Summary: The Tupolev Tu-160 (Russian: Туполев Ту-160 «Белый лебедь», romanized: Bely Lebed, lit. 'White Swan'; NATO reporting name: Blackjack) is a supersonic, variable-sweep wing nuclear-capable heavy strategic bomber and airborPage 160 doesn't appear to be a specific page that I can provide information on. Could you please provide more context or specify which page 160 you are referring to? I'll do my best to help.

> Finished chain.


{'input': 'Tell me about  page 160 ',
 'output': "Page 160 doesn't appear to be a specific page that I can provide information on. Could you please provide more context or specify which page 160 you are referring to? I'll do my best to help."}